In [188]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
import sklearn.metrics as metrics
import numpy as np

In [189]:
df = pd.read_csv('data/data_soja_cepea.csv')
df['data'] = pd.to_datetime(df['data'])
df = df.set_index('data')
# df = df.sort_index()
df

,preco_br
data,
1997-07-31,18.04
1997-08-01,17.97
1997-08-04,18.17
1997-08-05,18.10
1997-08-06,18.23
...,...
2021-03-08,164.85
2021-03-09,166.40
2021-03-10,169.58


### Feature Engineering

In [190]:
df2 = df[['preco_br']]

df2.loc[:, 'yesterday'] = df2.loc[:,'preco_br'].shift()
df2.loc[:, 'yesterday_dif'] = df2.loc[:, 'yesterday'].diff()

df2.loc[:, 'yesterday_1'] = df2.loc[:, 'yesterday'].shift()
# df2.loc[:, 'yesterday_1_dif'] = df2.loc[:, 'yesterday_1'].diff()

# df2.loc[:, 'last_week'] = df2.loc[:, 'yesterday'].shift(7)
# df2.loc[:, 'last_week_diff'] = df2.loc[:, 'last_week'].diff()


# df2.loc[:, 'last_year'] = df2.loc[:, 'yesterday'].shift(365)
# df2.loc[:, 'last_year_diff'] = df2.loc[:, 'last_year'].diff()
# df2.loc[:, 'last_year_diff'] = df2['yesterday'].sub(df2['last_year'], axis = 0) 

# df2.loc[:, 'last_month'] = df2.loc[:, 'yesterday'].shift(30)
# df2.loc[:, 'last_month_diff'] = df2.loc[:, 'last_month'].diff()

df2 = df2.dropna()
df2

,preco_br,yesterday,yesterday_dif,yesterday_1
data,,,,
1997-08-04,18.17,17.97,-0.07,18.04
1997-08-05,18.10,18.17,0.20,17.97
1997-08-06,18.23,18.10,-0.07,18.17
1997-08-07,18.16,18.23,0.13,18.10
1997-08-08,18.15,18.16,-0.07,18.23
...,...,...,...,...
2021-03-08,164.85,165.47,1.92,163.55
2021-03-09,166.40,164.85,-0.62,165.47
2021-03-10,169.58,166.40,1.55,164.85


In [191]:
df2.to_csv('data/final/soja_cepea_final.csv')

In [192]:
df2['2020-01']

,preco_br,yesterday,yesterday_dif,yesterday_1
data,,,,
2020-01-02,82.71,83.07,0.28,82.79
2020-01-03,82.80,82.71,-0.36,83.07
2020-01-06,82.92,82.80,0.09,82.71
2020-01-07,83.01,82.92,0.12,82.80
2020-01-08,82.83,83.01,0.09,82.92
2020-01-09,82.81,82.83,-0.18,83.01
2020-01-10,82.95,82.81,-0.02,82.83
2020-01-13,82.83,82.95,0.14,82.81
2020-01-14,82.83,82.83,-0.12,82.95


### Cross-Validation on Time Series 

In [193]:
year_list = df2.index.year.unique().tolist()
splits = {'train': [], 'test': []}

for idx, yr in enumerate(year_list[:-1]):
    train_yr = year_list[:idx+1]
    test_yr = [year_list[idx+1]]
    print('TRAIN: ', train_yr, 'TEST: ',test_yr)
    
    splits['train'].append(df2.loc[df2.index.year.isin(train_yr), :])
    splits['test'].append(df2.loc[df2.index.year.isin(test_yr), :])
    

TRAIN:  [1997] TEST:  [1998]
TRAIN:  [1997, 1998] TEST:  [1999]
TRAIN:  [1997, 1998, 1999] TEST:  [2000]
TRAIN:  [1997, 1998, 1999, 2000] TEST:  [2001]
TRAIN:  [1997, 1998, 1999, 2000, 2001] TEST:  [2002]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002] TEST:  [2003]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 2003] TEST:  [2004]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004] TEST:  [2005]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005] TEST:  [2006]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006] TEST:  [2007]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007] TEST:  [2008]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008] TEST:  [2009]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009] TEST:  [2010]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010] TEST:  [2011]
TRAIN:  [1997, 1998, 1999, 2000, 2001, 2002, 200

In [194]:
model = LinearRegression()

for i, yr in enumerate(year_list[:-1]):
    X_train = splits['train'][i].drop(['preco_br'], axis = 1).values # X
    y_train = splits['train'][i]['preco_br'].values # Y
    
    X_test =  splits['test'][i].drop(['preco_br'], axis = 1).values
    y_test = splits['test'][i]['preco_br'].values
    
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    
    r2 = metrics.r2_score(y_test, pred)
    print(r2)

0.9749100770755839
0.9840399752724416
0.9722249321885106
0.9958345454030079
0.9969869713299261
0.9890792003589725
0.9923834476129525
0.9555952898968967
0.9883449437932188
0.9966002104598737
0.9446849008054875
0.9705923731307794
0.9957336232523049
0.9623814755520954
0.9984189785686776
0.9886186905687223
0.9744471047903132
0.9927903329794346
0.9812146946037219
0.9684225579928661
0.9892504742675902
0.9889535476963727
0.9985422288254201
0.9061908252592047


### Testando a regressao com os dados aleatorizados

In [195]:
X = df2.drop(['preco_br'], axis = 1).values
y = df2.loc[:, 'preco_br']

In [196]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 42)

In [197]:
model_v1 = LinearRegression()
model_v1.fit(X_train, y_train)
predict_v1 = model_v1.predict(X_test)
metrics.r2_score(y_test, predict_v1)

0.9996096702433325